Подключаемся к гугл диску(Это понадобится для подгрузки данных для генерации видеоролика)


In [ ]:
import numpy as np
t_a = np.array([2500000000,
8100000000,
10000000000,
62500000000,
250000000000])

t_e = np.array([728016,
1367505,
1556017,
4047826,
8595635,
])

t_e/t_a

array([2.91206400e-04, 1.68827778e-04, 1.55601700e-04, 6.47652160e-05,
       3.43825400e-05])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -d -r Real-Time-Voice-Cloning

rm: cannot remove 'Real-Time-Voice-Cloning': No such file or directory


In [ ]:
#@title Setup CorentinJ/Real-Time-Voice-Cloning

#@markdown * clone the project
#@markdown * download pretrained models
#@markdown * initialize the voice cloning models

%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q --upgrade gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

  # download pretrained model
  !cd {project_name} && wget https://github.com/blue-fish/Real-Time-Voice-Cloning/releases/download/v1.0/pretrained.zip && unzip -o pretrained.zip
  !cd {project_name} && mkdir -p saved_models/default/
  
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1q8mEGwCkFy23KZsinbuvdKAQLqNKbYf1
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1EqFMIbvxffxtjiVrtykroF6_mUh-5Z3s
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1cf2NO6FtI0jDuy8AV3Xgn6leO6dHjIgu

import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

!ls 
encoder.load_model(project_name / Path("saved_models/default/encoder.pt"))
synthesizer = Synthesizer(project_name / Path("saved_models/default/synthesizer.pt"))
vocoder.load_model(project_name / Path("saved_models/default/vocoder.pt"))

ValueError: ignored

Необходимо перейти в content->Real-Time-Voice-Cloning->saved_models->default и вставить данные файлы с расширением .pt:

https://drive.google.com/uc?id=1q8mEGwCkFy23KZsinbuvdKAQLqNKbYf1

https://drive.google.com/uc?id=1EqFMIbvxffxtjiVrtykroF6_mUh-5Z3s

https://drive.google.com/uc?id=1cf2NO6FtI0jDuy8AV3Xgn6leO6dHjIgu


Запись или загрузка аудиофайлов. В панеле справа выберете способ загрузки аудиофайла. Выбрав "Upload", можео загрузить сразу несколько аудиозаписей.

In [ ]:
#@title Record or Upload
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav) 

SAMPLE_RATE = 22050
record_or_upload = "Upload (.mp3 or .wav)" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

embedding = None
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  global embedding
  embedding = None
  embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))
def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)

if record_or_upload == "Record":
  button = widgets.Button(description="Record Your Voice")
  button.on_click(_record_audio)
  display(button)
else:
  #button = widgets.Button(description="Upload Voice File")
  #button.on_click(_upload_audio)
  _upload_audio("")

Saving NoSilenceAXxnsJaFCic_44100_1_short_1.wav to NoSilenceAXxnsJaFCic_44100_1_short_1 (2).wav
Saving NoSilenceAXxnsJaFCic_44100_1_short_2.wav to NoSilenceAXxnsJaFCic_44100_1_short_2 (2).wav
Saving NoSilenceAXxnsJaFCic_44100_4_1_short_1.wav to NoSilenceAXxnsJaFCic_44100_4_1_short_1 (2).wav
Saving NoSilenceAXxnsJaFCic_44100_4_1_short_2.wav to NoSilenceAXxnsJaFCic_44100_4_1_short_2 (2).wav


Синтезирование голоса.

In [ ]:
#@title Synthesize a text { run: "auto" }
text = "Is this the real life? Is this just fantasy? Caught in a landside, No escape from reality Open your eyes, Look up to the skies and see" #@param {type:"string"}
def synthesize(embed, text):
  print("Synthesizing new audio...")
  #with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  clear_output()
  display(Audio(generated_wav, rate=synthesizer.sample_rate, autoplay=True))
  return generated_wav

if embedding is None:
  print("first record a voice or upload a voice file!")
else:
  baiden_waw = synthesize(embedding, text)

# **Wav2Lip**

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!git clone https://github.com/Rudrabha/Wav2Lip.git

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 360, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 360 (delta 11), reused 20 (delta 11), pack-reused 338
Receiving objects: 100% (360/360), 522.85 KiB | 13.41 MiB/s, done.
Resolving deltas: 100% (195/195), done.


Загрузить файл

https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fradrabha%5Fm%5Fresearch%5Fiiit%5Fac%5Fin%2FDocuments%2FWav2Lip%5FModels%2Fwav2lip%5Fgan%2Epth&parent=%2Fpersonal%2Fradrabha%5Fm%5Fresearch%5Fiiit%5Fac%5Fin%2FDocuments%2FWav2Lip%5FModels&ga=1

себе на гугл диск. В папку MyDrive->Wav2lip сохраняем данный файл.
В скобочках вставить полный путь к файлу.

Папки colab/deepfake необязательно создавать, созданы для удобства использования.

In [ ]:
!cp -ri "/content/drive/MyDrive/colab/deepfake/Wav2lip/wav2lip_gan.pth" /content/Wav2Lip/checkpoints/

In [ ]:
!pip uninstall tensorflow tensorflow-gpu

Found existing installation: tensorflow 1.15.2
Uninstalling tensorflow-1.15.2:
  Would remove:
    /tensorflow-1.15.2/python3.7/tensorflow-1.15.2.dist-info/*
    /tensorflow-1.15.2/python3.7/tensorflow/*
    /tensorflow-1.15.2/python3.7/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.2


In [ ]:
!cd Wav2Lip && pip install -r requirements.txt

     |████████████████████████████████| 1.6 MB 13.2 MB/s 
     |████████████████████████████████| 20.3 MB 10.2 MB/s 
     |████████████████████████████████| 66.7 MB 43 kB/s 
     |████████████████████████████████| 26.6 MB 1.3 MB/s 
     |████████████████████████████████| 676.9 MB 3.6 kB/s 
     |████████████████████████████████| 2.6 MB 44.7 MB/s 
     |████████████████████████████████| 60 kB 8.0 MB/s 
     |████████████████████████████████| 3.5 MB 40.7 MB/s 
     |████████████████████████████████| 20.2 MB 53.1 MB/s 
  Created wheel for librosa: filename=librosa-0.7.0-py3-none-any.whl size=1598360 sha256=211a7e290d2bb0def87d02d6eac579b4317619c6a71e2daa0b4361a62149a479
  Stored in directory: /root/.cache/pip/wheels/f5/0d/b1/71a4e1863d7577a218f771df59c7582996580fa07ac79d68fd
Successfully built librosa
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: llvmlite
    Fo

In [ ]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

--2022-04-24 09:54:20--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’

Wav2Lip/face_detect 100%[===================>]  85.68M   150MB/s    in 0.6s    

2022-04-24 09:54:21 (150 MB/s) - ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’ saved [89843225/89843225]



Сохранение wav файла в папку Wav2Lip. С этой папки будет подгружена аудиозапись для создания видеоролика.

In [ ]:
from scipy.io.wavfile import write
samplerate = 16000
write(f"/content/drive/MyDrive/colab/deepfake/Wav2Lip/baiden.wav", samplerate, baiden_waw)

В папке MyDrive создаём папку Wav2Lip и загружаем туда видеозапись и ранее записанный аудиофайл(нужно указать путь).

In [ ]:
!cp "/content/drive/MyDrive/colab/deepfake/Wav2Lip/baidenVideo.mp4" "/content/drive/MyDrive/colab/deepfake/Wav2Lip/baiden.wav" sample_data/
!ls sample_data/

anscombe.json		      mnist_test.csv
baidenVideo.mp4		      mnist_train_small.csv
baiden.wav		      nhP5buMAZ7w_44100_9_10_middle.wav
california_housing_test.csv   Putin_chose_this_war_Biden_says.mp4
california_housing_train.csv  README.md


In [ ]:
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/baidenVideo.mp4" --audio "../sample_data/baiden.wav" --resize_factor 2

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 3690
(80, 925)
Length of mel chunks: 274
  0% 0/3 [00:00<?, ?it/s]
  0% 0/18 [00:00<?, ?it/s]
  6% 1/18 [00:43<12:13, 43.13s/it]
 11% 2/18 [01:21<11:07, 41.70s/it]
 17% 3/18 [01:59<10:09, 40.66s/it]
 22% 4/18 [02:38<09:20, 40.05s/it]
 28% 5/18 [03:17<08:35, 39.64s/it]
 33% 6/18 [03:55<07:51, 39.25s/it]
 39% 7/18 [04:33<07:09, 39.02s/it]
 44% 8/18 [05:12<06:29, 38.95s/it]
 50% 9/18 [05:51<05:49, 38.80s/it]
 56% 10/18 [06:29<05:09, 38.65s/it]
 61% 11/18 [07:08<04:31, 38.71s/it]
 67% 12/18 [07:46<03:51, 38.54s/it]
 72% 13/18 [08:24<03:12, 38.56s/it]
 78% 14/18 [09:03<02:34, 38.56s/it]
 83% 15/18 [09:41<01:55, 38.51s/it]
 89% 16/18 [10:25<01:20, 40.08s/it]
 94% 17/18 [11:05<00:39, 39.87s/it]
100% 18/18 [11:09<00:00, 37.22s/it]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100% 3/3 [11:55<00:00, 238.41s/it]
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers


/content/Wav2Lip/results в данной директории будет лежать созданный видеоролик.